In [13]:
import csv
import os
import random
import warnings
from music21 import stream, note, duration, pitch
from music21.musicxml import m21ToXml

# Suppress annoying MusicXMLWarning
warnings.filterwarnings("ignore", category=m21ToXml.MusicXMLWarning)

# Your pitch-to-label mapping
pitch_dict = {
    'B3': -8, 'C4': -7, 'D4': -6, 'E4': -5, 'F4': -4, 'G4': -3, 'A4': -2, 'B4': -1, 'C5': 0,
    'D5': 1, 'E5': 2, 'F5': 3, 'G5': 4, 'A5': 5, 'B5': 6, 'C6': 7, 'D6': 8
}

def generate_random_note():
    '''
    A function to generate a random note based on a predefined list
    of pitches and durations.
    '''
    pitches = list(pitch_dict.keys())
    durations = ['whole', 'half', 'quarter', 'eighth', '16th']

    selected_pitch = random.choice(pitches)
    selected_duration = random.choice(durations)

    n = note.Note()
    n.pitch = pitch.Pitch(selected_pitch)
    n.duration = duration.Duration(selected_duration)

    return n, selected_pitch, selected_duration

def generate_music(num_sheets=5, num_notes=50, output_folder='../raw_data/musicxml_files', label_file='../raw_data/labels.csv'):
    '''
    A function to create musicXML files with a specified number of random notes,
    and generate a label file with the note values.
    '''
    current_dir = os.getcwd()
    output_folder = os.path.abspath(os.path.join(current_dir, output_folder))
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    label_file = os.path.abspath(os.path.join(current_dir, label_file))

    with open(label_file, 'w', newline='') as csvfile:
        label_writer = csv.writer(csvfile)
        label_writer.writerow(['filename', 'labels'])

        for i in range(num_sheets):
            s = stream.Stream()
            note_values = []

            for _ in range(num_notes):
                n, selected_pitch, selected_duration = generate_random_note()

                # Ensure no tied notes
                if len(s.notes) > 0 and s.notes[-1].pitch == n.pitch and s.notes[-1].duration == n.duration:
                    continue

                s.append(n)
                note_values.append(pitch_dict[selected_pitch])

            filename = f'music_{i}.musicxml'
            s.write('musicxml', fp=os.path.join(output_folder, filename))
            label_writer.writerow([filename.replace('.musicxml', '.png'), note_values])

    return None


In [2]:
import subprocess
import platform
import os


# get the right path for musescore based on system
def get_musescore_path():
    system = platform.system()
    if system == 'Windows':
        return r'C:\Program Files\MuseScore 4\bin\MuseScore4.exe'  # Update this path if necessary
    elif system == 'Darwin':  # macOS
        return '/Applications/MuseScore 4.app/Contents/MacOS/mscore'
    elif system == 'Linux':
        return '/usr/bin/musescore4'  # Update this path if necessary
    else:
        raise ValueError("Unsupported operating system")


def convert_musicxml_to_png(input_folder='../raw_data/musicxml_files', output_folder='../raw_data/sheet_images'):
    current_dir = os.getcwd()
    input_folder = os.path.abspath(os.path.join(current_dir, input_folder))
    output_folder = os.path.abspath(os.path.join(current_dir, output_folder))
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    musescore_path = get_musescore_path()

    for file_name in os.listdir(input_folder):
        if file_name.endswith('.musicxml'):
            input_path = os.path.join(input_folder, file_name)
            output_filename = file_name.replace('.musicxml', '.png')
            output_path = os.path.join(output_folder, output_filename)
            result = subprocess.run([musescore_path, input_path, '-o', output_path], stderr=subprocess.PIPE)
            if result.returncode != 0:
                print(f"Error processing {file_name}: {result.stderr.decode('utf-8')}")

    return None

In [8]:
# Example usage
generate_music()
convert_musicxml_to_png()